# Demo Notebook

Steps:
1. Download SAE with SAE Lens.
2. Create a dataset consistent with that SAE. 
3. Fold the SAE decoder norm weights so that feature activations are "correct".
4. Estimate the activation normalization constant if needed, and fold it into the SAE weights.
5. Run the SAE generator for the features you want.

# Set Up

In [1]:
import torch
from sae_lens import SAE 
from transformer_lens import HookedTransformer
from sae_dashboard.sae_vis_data import SaeVisConfig
from sae_dashboard.sae_vis_runner import SaeVisRunner

/home/paperspace/SAEDashboard/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1. Download / Initialize SAE

In [8]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

model = HookedTransformer.from_pretrained("gemma-2b", device = device)

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience. 
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-2b-res-jb", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = "blocks.6.hook_resid_post", # won't always be a hook point
    device = device
)
# fold w_dec norm so feature activations are accurate
sae.fold_W_dec_norm()



Device: cuda


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


Loaded pretrained model gemma-2b into HookedTransformer


# 2. Get token dataset

In [9]:
from sae_lens import ActivationsStore

activations_store = ActivationsStore.from_sae(
    model = model,
    sae = sae,
    streaming=True,
    store_batch_size_prompts=8,
    n_batches_in_buffer=8,
)

# Some SAEs will require we estimate the activation norm and fold it into the weights. This is easy with SAE Lens. 
if sae.cfg.normalize_activations == "expected_average_only_in":
    norm_scaling_factor = activations_store.estimate_norm_scaling_factor(n_batches_for_norm_estimate=30)
    sae.fold_activation_norm_scaling_factor(norm_scaling_factor)

In [10]:
from tqdm import tqdm 

def get_tokens(
    activations_store: ActivationsStore,
    n_batches_to_sample_from: int = 4096 * 6,
    n_prompts_to_select: int = 4096 * 6,
):
    all_tokens_list = []
    pbar = tqdm(range(n_batches_to_sample_from))
    for _ in pbar:
        batch_tokens = activations_store.get_batch_tokens()
        batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][
            : batch_tokens.shape[0]
        ]
        all_tokens_list.append(batch_tokens)

    all_tokens = torch.cat(all_tokens_list, dim=0)
    all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
    return all_tokens[:n_prompts_to_select]

# 1000 prompts is plenty for a demo.
token_dataset = get_tokens(activations_store, 128, 128)

100%|██████████| 128/128 [00:03<00:00, 35.79it/s]


# Step 3 Evaluate the SAE

In [11]:
from sae_lens import run_evals

eval_metrics = run_evals(
    sae=sae,
    activation_store = activations_store,
    model = model,
    n_eval_batches=3,
    eval_batch_size_prompts=8
)

# CE Loss score should be high for residual stream SAEs
print(eval_metrics["metrics/CE_loss_score"])

# ce loss without SAE should be fairly low < 3.5 suggesting the Model is being run correctly
print(eval_metrics["metrics/ce_loss_without_sae"])

# ce loss with SAE shouldn't be massively higher
print(eval_metrics["metrics/ce_loss_with_sae"])

0.9686041673024496
1.9312611818313599
2.260663708051046


# 4. Generate Feature Dashboards

In [14]:
from pathlib import Path
test_feature_idx_gpt = list(range(256))

feature_vis_config_gpt = SaeVisConfig(
    hook_point=sae.cfg.hook_name,
    features=test_feature_idx_gpt,
    minibatch_size_features=16,
    minibatch_size_tokens=16, # this is really prompt with the number of tokens determined by the sequence length
    verbose=True,
    device="cuda",
    cache_dir=Path("demo_activations_cache"), # this will enable us to skip running the model for subsequent features.
)

data = SaeVisRunner(feature_vis_config_gpt).run(
    encoder=sae, # type: ignore
    model=model,
    tokens=token_dataset,
)

┏━━━━━━┳━━━━━━┳━━━━━━━┓
┃ Task ┃ Time ┃ Pct % ┃
┡━━━━━━╇━━━━━━╇━━━━━━━┩
└──────┴──────┴───────┘

Extracting vis data from cached data: 100%|██████████| 256/256 [00:24<00:00, 10.50it/s]

Extracting vis data from cached data: 100%|██████████| 256/256 [00:24<00:00, 10.50it/s]


In [15]:
from sae_dashboard.data_writing_fns import save_feature_centric_vis
filename = f"demo_feature_dashboards.html"
save_feature_centric_vis(sae_vis_data=data, filename=filename)



Forward passes to cache data for vis:   0%|          | 0/128 [01:18<?, ?it/s]



































































































Saving feature-centric vis: 100%|██████████| 256/256 [00:04<00:00, 56.58it/s]
